# Kappel et al, 2021
## Figure 1: c-fos activity mapping

Panels
- c
- d
- e

### running this notebook requires 128 GB RAM (96 GB might be sufficient)


In [ ]:
import nrrd
import tifffile as tiff
import numpy as np
import glob, os
import matplotlib.pyplot as plt
import colorcet as cc
import matplotlib.gridspec as gridspec
from matplotlib.colors import LinearSegmentedColormap
import pandas as pd
import seaborn as sns

from matplotlib import lines
from scipy import ndimage as nd
import skimage.measure
import matplotlib.image as mpimg
import notebookHelper as nh
import scipy.stats as sta
import h5py
import matplotlib.patches as mpatches
import paperFigureProps as pfp

import cv2 as cv2
pfp.paper()
inToCm=2.54


In [ ]:
if 'startDirMaster' not in locals():
    startDirMaster=os.getcwd()

propsFn=startDirMaster+'\\props.csv'

props=pd.read_csv(propsFn, header=None, index_col=0, squeeze=True,delim_whitespace=True).to_dict()
#print(props)

base=props['BaseDir']
expFile=props['allExpBehFn']
anFile=props['allAnBehFn']

RawDataDir = os.path.join(base,props['RawDataDir'])+'\\'+'HCR\\'
ProcessingDir = os.path.join(base,props['ProcessingDir'])+'\\'
outputDir = os.path.join(base,props['outputDir'])+'\\'
FigureDir = os.path.join(base,props['FigureDir'])+'\\'


if not os.path.isdir(ProcessingDir):
    os.makedirs(ProcessingDir)

if not os.path.isdir(outputDir):
    os.makedirs(outputDir)
    
if not os.path.isdir(FigureDir):
    os.makedirs(FigureDir)

os.chdir('..\\')
print('base',base)
print('RawDataDir',RawDataDir)
print('ProcessingDir',ProcessingDir)
print('FigureDir',FigureDir)

In [ ]:
#propsFn='props.csv'
#props=pd.read_csv(propsFn, header=None, index_col=0, squeeze=True,delim_whitespace=True).to_dict()

#base=props['BaseDir']
#artDir=props['ArtDir']
#outputDir = props['outputDir']
#RawDataDir = props['RawDataDir']+'HCR\\'
#ProcessingDir = props['ProcessingDir']
#outputDir = props['outputDir']
#FigureDir=props['FigureDir']

#expFile=props['allExpBehFn']
#anFile=props['allAnBehFn']

p_ref_dorsalHCR=RawDataDir+'dorsal\\juvenile_hcr_ref_dv_d.tif'
p_ref_ventralHCR=RawDataDir+'ventral\\juvenile_hcr_ref_dv_v_bspline_aligned.tif'
p_dorsalStacks=RawDataDir+'dorsal\\aligned\\'
p_ventralStacks=RawDataDir+'ventral\\aligned\\to_dorsal\\'
p_dv_correspondence=RawDataDir+'hcr_d_v_correspondence.csv'
p_cellCounts=RawDataDir+'CellCounts_summary.xlsx'
#p_masks=props['RawDataDir']+'masks\\2pmasks_hcr_ref_dv_merge_rev.nrrd'
p_masks=os.path.join(base,props['RawDataDir'])+'\\'+'masks\\2pmasks_hcr_ref_dv_merge_rev_20210518b.nrrd'
p_masks_cFos=os.path.join(base,props['RawDataDir'])+'\\'+'masks\\hcrSuperStackMasks_20210716b.nrrd'
p_maskComments=os.path.join(base,props['RawDataDir'])+'\\'+'masks\\MaskCommentsJL.xlsx'

p_h5pyAll=RawDataDir+'allMerge.h5'
rereadIms=True


titles=np.array(['Bout-like','Continuous','Conspecific','No stim'])[[3,1,0,2]]

props

In [ ]:
#Colors for no-stim, continuous, bout-like, conspecific

co=sns.color_palette("tab10", 4)
co[1:]=co[:-1]
co[0]=[.6,.6,.6]


In [ ]:
def read_good_nrrd(f):
    tmp=nrrd.read(f)
    im = tmp[0].astype(np.uint8)
    im = np.moveaxis(im, 2, 0)
    im = np.moveaxis(im, 1, 2)
    return im

def clip_stack(stack, lims=(1, 99)):
    
    zmin, zmax = np.nanpercentile(stack, lims[0]), np.nanpercentile(stack, lims[1])
    print(zmin, zmax)
    stack[stack<zmin] = zmin
    stack[stack>zmax] = zmax

    return stack.astype(np.uint8)

In [ ]:
def pseudoOrtho(img, clim=None, lims=None):
    ## plot mean projections xy,zy, zx of img next to each other
    fig,axes=plt.subplots(2,2,
                          gridspec_kw={'height_ratios': [3, 1],
                                      'width_ratios': [1, 1]},
                         figsize=(4,4))
    axes=axes.ravel()
    
    if lims is None:

        axes[0].imshow(np.nanmean(img,axis=0),clim=clim)
        axes[1].imshow(np.nanmean(img,axis=2).T,clim=clim)
        axes[2].imshow(np.nanmean(img,axis=1),clim=clim)
    else:
        axes[0].imshow(np.nanmean(img[lims[0][0]:lims[0][1],:,:],axis=0),clim=clim)
        axes[0].axhline(lims[1][0]+50)
        axes[0].axvline(lims[2][0]+50)
        
        axes[1].imshow(np.nanmean(img[:,:,lims[2][0]:lims[2][1]],axis=2).T,clim=clim)
        axes[2].imshow(np.nanmean(img[:,lims[1][0]:lims[1][1],:],axis=1),clim=clim)
        
    axes[1].set_xlim(0,450)
    
    for a in axes:
        a.axis('off')
        a.set_xticks([])
        a.set_yticks([])
        a.set_ylabel("")
        a.set_position([0, 0, 1, 1], which='both')
        
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=None)
    fig.tight_layout(pad=.5)


# merging of dorsal and ventral HCR data
## start with aligned averages

In [ ]:
ref_d=tiff.imread(p_ref_dorsalHCR)
ref_v=tiff.imread(p_ref_ventralHCR)

In [ ]:
pseudoOrtho(ref_d)

In [ ]:
pseudoOrtho(ref_v)

In [ ]:
ref_merge_avg=(ref_d+ref_v)/2.
pseudoOrtho(ref_merge_avg)

In [ ]:
masks=read_good_nrrd(p_masks)
pseudoOrtho(masks)

In [ ]:
masks_cFos=read_good_nrrd(p_masks_cFos)
pseudoOrtho(masks_cFos)

## Now, merge dorsal and ventral brains for each animal

In [ ]:
# External table lists corresponding file names for d and v brains
# note: 4 ventral brains are missing. not using the dorsal half either. These are excluded later.
dvCorr=pd.read_csv(p_dv_correspondence,index_col=0)
dvCorr.head()

In [ ]:
groups = ['noStim', 'cont', 'bout', 'conspecific']
animals=['{0:02d}'.format(a+1) for a in range(8)]

if rereadIms:

    ref=ref_merge_avg
    groups = ['noStim', 'cont', 'bout', 'conspecific']
    animals=['{0:02d}'.format(a+1) for a in range(8)]

    cref = 'C1'
    cort = 'C2'
    cfos = 'C3'

    channels = [cref, cort, cfos]

    ims_all=np.zeros(shape=(len(groups),8,len(channels), ref.shape[0], ref.shape[1], ref.shape[2]), dtype=np.uint8).astype('float')

    for index,row in dvCorr.iterrows():

        group=row.group
        groupNr=row.groupNr
        animal=row.animal
        d=row.d
        v=row.v

        for c,channel in enumerate(channels):

            print(channel, group,groupNr, animal)
            print(d)
            print(v)

            if (v=='none'):
                im_merge_avg=im_d*np.nan
            else:
                fn_d = p_dorsalStacks+channel+"-"+d
                fn_v = p_ventralStacks+channel+"-"+v
                im_d=read_good_nrrd(fn_d)[::-1,:,:]
                im_v=read_good_nrrd(fn_v)
                im_merge_avg=(im_d+im_v)/2.

            ims_all[groupNr,animal,c]=im_merge_avg

            pseudoOrtho(ims_all[groupNr,animal,c])
            plt.show()


In [ ]:
if rereadIms:
    h5f = h5py.File(p_h5pyAll, 'w')
    #h5f.create_dataset('ims_all', data=ims_all) #uncomment to save all combined imaged. takes time!
    h5f.close()
else:
    h5f = h5py.File(p_h5pyAll,'r') 
    ims_all = h5f['ims_all'][:]
    h5f.close()

In [ ]:
ims_all.shape

## generate merged channel arrays for some sub groups for convenience

In [ ]:
refAllMerge=np.zeros(np.array(ims_all.shape)[[0,3,4,5]],dtype='uint8')

#note: successive averaging to avoid huge interim variable (~40GB RAM)
for i in range(4):
    refAllMerge[i]=np.nanmean(ims_all[i,:,0],axis=0)
    
print(refAllMerge.shape)
refAllMerge=np.nanmean(refAllMerge,axis=0)
print(refAllMerge.shape)

In [ ]:
if rereadIms:
    pass
    #tiff.imsave(RawDataDir+'refAllMerge.tif',refAllMerge)
#else:
#    cfosMergeCS=tiff.imread(RawDataDir+'cfosMergeCS.tif')
#    refAllMerge=tiff.imread(RawDataDir+'refAllMerge.tif')

# Plot overview mean  z- projections of raw fluorescence
### Not used in paper
### testing various normalization strategies for cFos

In [ ]:
def plot_zProject_groups(img,
                         groups,
                         clim_c1=(0,15),
                         clim_c2=(0,5),
                         clim_c3=(0,3),
                        xlim=None,
                        ylim=None,
                        zlim=(0,-1),
                        norm=False):
    
    #thalamus:  s2 = 247-75, s1 = 247-92
    s1=zlim[0]
    s2=zlim[1]
    fig = plt.figure(figsize=(12, 12), dpi=200)

    gs = gridspec.GridSpec(nrows=3, ncols=5, width_ratios=[1, 1, 1, 1, .1], height_ratios=[1, 1, 1])
    gs.update(wspace=0.1, hspace=0.1)



    cmap_cort = LinearSegmentedColormap.from_list('cmap_cort', ['black', 'magenta'])
    cmap_cort = 'inferno'

    for gno, group in enumerate(groups):

        ax1 = plt.subplot(gs[0, gno])
        ax2 = plt.subplot(gs[1, gno])
        ax3 = plt.subplot(gs[2, gno])

        c1 = np.nanmean(img[gno,:,0],axis=0)
        c2 = np.nanmean(img[gno,:,1],axis=0)
        c3 = np.nanmean(img[gno,:,2],axis=0)
        
        #normalization attempts 1-3 were trying to deal with division by very low numbers 
        # were the reference channel has little signal. Abandoned these approaches
        
        if norm==1: # norm by masked reference channel where ref channel signal above threshold (not good)
            c3Norm=c3.copy()
            normMask=c1>.5
            c3Norm[normMask]=c3Norm[normMask]/c1[normMask]
            c3=c3Norm*normMask      
            
        elif norm==2:
            c3=c3/(c1+1)
            
        elif norm==3:
            c3=c3/(c1+.1)
            
        #Instead, divide by 3D gaussian smoothed ref channel. This is used in figures showing imaging planes
            
        elif norm==4: # divide all channels
            c1Filt=nd.gaussian_filter(c1,(5,20,20))
            c3=c3/c1Filt
            c1=c1/c1Filt
            c2=c2/c1Filt
            
        elif norm==5: # as in 4, showing filtered ref instead of ref for illustration
            c1Filt=nd.gaussian_filter(c1,(5,20,20))
            c3=c3/c1Filt
            c1=c1Filt
            c2=c2/c1Filt
            
        elif norm==6: # as in 5, masking non-brain areas
            c1Filt=nd.gaussian_filter(c1,(5,20,20))
            maskFilt=c1Filt>.3
            c3=maskFilt*c3/c1Filt
            c1=maskFilt*c1/c1Filt
            c2=maskFilt*c2/c1Filt

        elif norm==7: # simple division, OK with floats - this approach is later used for quantification

            c3=c3/c1
            c2=c2/c1
            c1=c1

        ax1.imshow(c1[s1:s2].mean(axis=0), clim=clim_c1, cmap='viridis')
        ax2.imshow(c3[s1:s2].mean(axis=0), clim=clim_c3, cmap='viridis')
        ax3.imshow(c2[s1:s2].mean(axis=0), clim=clim_c2, cmap=cmap_cort)

        for a in [ax1, ax2, ax3]:
            a.set_axis_off()
            if xlim!=None:
                a.set_xlim(xlim[0],xlim[1])
            if ylim!=None:
                a.set_ylim(ylim[0],ylim[1])

        ax1.set_title(group)

    ax4 = plt.subplot(gs[0, 4])
    sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=clim_c1[0], vmax=clim_c1[1]))
    sm._A = []
    clb = plt.colorbar(sm, cax=ax4, use_gridspec=True, label='Intensity, a.u.', pad=10)

    ax5 = plt.subplot(gs[1, 4])
    sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=clim_c3[0], vmax=clim_c3[1]))
    sm._A = []
    clb = plt.colorbar(sm, cax=ax5, use_gridspec=True, label='Intensity, a.u.', pad=10)

    ax6 = plt.subplot(gs[2, 4])
    sm = plt.cm.ScalarMappable(cmap=cmap_cort, norm=plt.Normalize(vmin=clim_c2[0], vmax=clim_c2[1]))
    sm._A = []
    clb = plt.colorbar(sm, cax=ax6, use_gridspec=True, label='Intensity, a.u.', pad=10)

    plt.show()

In [ ]:
# define z-planes of thalamus
s2 = 247-75
s1 = 247-92

In [ ]:

cortX1,cortX2=190,215 #area of thalamus cluster in x,y
cortY1,cortY2=300,315
cortProfile_all=[]
for gr in range(4):
    for an in range(8):
        cortProfile_all.append(np.mean(ims_all[gr,an,1,:,cortY1:cortY2,cortX1:cortX2],axis=1).mean(axis=1))
        
cortProfile_all=np.array(cortProfile_all)     

fig,axes=plt.subplots(figsize=(3,2))
cols=np.array(['r','b','g','k'])
cols=cols[np.repeat(np.arange(4),8)]

for i in range(32):
    plt.plot(cortProfile_all[i]+i*10,c=cols[i]);
    
plt.axvline(s1)
plt.axvline(s2)
sns.despine()
plt.title('cort alignment (z-profile of cort signal)')

In [ ]:
# full stack z-projections, raw fluorescence
plot_zProject_groups(ims_all,groups)

# Thalamus planes mean z- projections of raw fluorescence

In [ ]:
# thalamic planes z-projection, raw fluorescence
plot_zProject_groups(ims_all,groups,zlim=(s1,s2))

In [ ]:
#Thalamic planes, normalized by filtered ref channel

plot_zProject_groups(ims_all,
                     groups,
                     zlim=(s1,s2),
                     clim_c1=(0,3),
                     clim_c2=(0,5),
                     clim_c3=(0,1),
                     norm=6)

In [ ]:
#Thalamic planes, normalized by  ref channel

plot_zProject_groups(ims_all,
                     groups,
                     zlim=(s1,s2),
                     clim_c1=(0,10),
                     clim_c2=(0,5),
                     clim_c3=(0,1),
                     norm=7)

# Thalamus zoom in
## normalized intensity

In [ ]:
#norm by 3d smoothed gaussian filtered ref.
plot_zProject_groups(ims_all,
                     groups,
                     zlim=(s1,s2),
                     xlim=(300,160),
                     ylim=(380,220),
                     clim_c1=(0,2),
                     clim_c2=(0,20),
                     clim_c3=(0,1.5),
                     norm=4)

In [ ]:
#norm by 3d smoothed gaussian  ref.
plot_zProject_groups(ims_all,
                     groups,
                     zlim=(s1,s2),
                     xlim=(300,160),
                     ylim=(380,220),
                     clim_c1=(0,10),
                     clim_c2=(0,20),
                     clim_c3=(0,1.5),
                     norm=7)

In [ ]:
# all individual animal max projections, cfos channel
fig,axes=plt.subplots(4,8,figsize=(15,10),sharex=True,sharey=True,dpi=200)
for an in range(8):
    for gr in range(4):
        ax=axes[gr,an]
        ax.imshow(np.mean(ims_all[gr,an,2,s1:s2,:,:],axis=0),clim=[0,10])
plt.tight_layout()

In [ ]:
# all individual animal max projections, reference channel
fig,axes=plt.subplots(4,8,figsize=(15,10),sharex=True,sharey=True,dpi=200)

for gr in range(4):
    for an in range(8):

        ax=axes[gr,an]
        ax.imshow(np.mean(ims_all[gr,an,0,s1:s2,:,:],axis=0),clim=[0,10])
plt.tight_layout()

In [ ]:
# all individual animal max projections, cort channel
fig,axes=plt.subplots(4,8,figsize=(15,10),sharex=True,sharey=True,dpi=200)
for an in range(8):
    for gr in range(4):
        ax=axes[gr,an]
        ax.imshow(np.mean(ims_all[gr,an,1,s1:s2,:,:],axis=0),clim=[0,10])
plt.tight_layout()

In [ ]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

In [ ]:
# all individual animal max projections, reference channel
# This step requires another ~40GB of RAM. Needs float datatype to accomodate the nans.

fig,axes=plt.subplots(4,8,figsize=(15,10),sharex=True,sharey=True,dpi=200)

fosNorm_all=np.zeros(shape=(4,8,*ims_all.shape[3:]),dtype='uint8').astype('float')
for gr in range(4):
    for an in range(8):
        
        c1 = ims_all[gr,an,0]*1.
        c3 = ims_all[gr,an,2] *1.
        c1Filt=nd.gaussian_filter(c1,(5,20,20))
        c3=c3/c1Filt
        fosNorm_all[gr,an]=c3

        ax=axes[gr,an]
        ax.imshow(np.nanmean(c3[s1:s2,:,:],axis=0),clim=[0,.5])
plt.tight_layout()


In [ ]:
plt.imshow(np.nanmean(fosNorm_all[3,0,s1:s2,:,:],axis=0),clim=[0,.5])

In [ ]:
gr=0
an=1

c1 = ims_all[gr,an,0]*1.
c3 = ims_all[gr,an,2] *1.
c1Filt=nd.gaussian_filter(c1,(5,10,10))
c3=c3/c1Filt
#fosNorm_all[gr,an]=c3

plt.imshow(np.nanmean(c1Filt[s1:s2,:,:],axis=0),clim=[0,5])

In [ ]:
# z-profile of ref signal around thalamus region
cortX1,cortX2=190,215
cortY1,cortY2=300,315
cortProfile_all=[]
for gr in range(4):
    for an in range(8):

        cortProfile_all.append(np.mean(ims_all[gr,an,0,:,cortY1:cortY2,cortX1:cortX2],axis=1).mean(axis=1))
        
cortProfile_all=np.array(cortProfile_all)     


fig,axes=plt.subplots(figsize=(15,10))
cols=np.array(['r','b','g','k'])
cols=cols[np.repeat(np.arange(4),8)]

for i in range(32):
    plt.plot(cortProfile_all[i]+i*5,c=cols[i]);
    
plt.axvline(s1)
plt.axvline(s2)
plt.title('ref alignment (z-profile of ref signal)')

colNames=['group','plane','cort']
print(colNames)
m,n = cortProfile_all.shape
tmp_arr = np.column_stack((np.repeat(np.arange(m),n),np.tile(np.arange(n),m),cortProfile_all.reshape(m*n,-1)))
cort_df = pd.DataFrame(tmp_arr,columns=colNames)
cort_df['condition']=np.repeat(np.array(groups),8)[cort_df.group.astype('int')]
print(cort_df.head())
fig,axes=plt.subplots(figsize=(15,10))

cort_df.groupby(['condition','plane']).cort.mean().unstack().T.plot(ax=axes);
plt.axvline(s1)
plt.axvline(s2)

# all intensity histograms, reference channel



In [ ]:

#masks = read_good_nrrd(p_masks)
brightMask=masks_cFos.copy().astype('float')
brightMask[brightMask==0]=np.nan
brightMask[brightMask==14]=np.nan
pseudoOrtho(brightMask)

In [ ]:
cortInt=ims_all[:,:,1,:,:].astype('uint8')
cort_merge_avg=np.nanmean(np.nanmean(cortInt,axis=0),axis=0)

In [ ]:
cort_merge_avg.dtype

In [ ]:
#generating convenience averages

In [ ]:
fosNorm_merge_avg=np.nanmean(fosNorm_all,axis=1)

In [ ]:
fosNorm_merge_avgavg=np.nanmean(fosNorm_merge_avg,axis=0)*100
fosNorm_merge_avgavg[fosNorm_merge_avgavg>255]=255

In [ ]:
fosNorm_merge_avgmax=np.nanmax(fosNorm_merge_avg,axis=0)*100
fosNorm_merge_avgmax[fosNorm_merge_avgmax>255]=255

In [ ]:
pseudoOrtho((masks[s1:s2]==4)*np.nanmean(fosNorm_all[1,:,s1:s2],axis=0),clim=[0,.3])

In [ ]:
#get intensities in each activity cluster from individuals
nAreas=masks_cFos.max()
fosMeans=np.zeros(shape=(4,8,nAreas))
refMeans=np.zeros(shape=(4,8,nAreas))
cortMeans=np.zeros(shape=(4,8,nAreas))
fosNormMeans=np.zeros(shape=(4,8,nAreas))
for gr in range(4):
    for an in range(8):
        for area in range(nAreas):
            maskNow=masks_cFos==area+1
            imgNow=ims_all[gr,an,2,:,:,:]
            datNow=imgNow[maskNow]
            refNow=ims_all[gr,an,0,:,:,:]
            datRefNow=refNow[maskNow]
            cortNow=ims_all[gr,an,1,:,:,:]
            datCortNow=cortNow[maskNow]
            fosNormNow=fosNorm_all[gr,an,:,:,:]
            datFosNormNow=fosNormNow[maskNow] # this is what gets used later in quantification
            fosMeans[gr,an,area]=np.nanmean(datNow)
            refMeans[gr,an,area]=np.nanmean(datRefNow)
            cortMeans[gr,an,area]=np.nanmean(datCortNow)
            fosNormMeans[gr,an,area]=np.nanmean(datFosNormNow)# this is what gets used later in quantification

In [ ]:
areaStr=['area.'+'{0:02d}'.format(a+1) for a in range(nAreas)]
colNames=['group','animal']
colNames.extend(areaStr)
colNames

In [ ]:
m,n,r = fosMeans.shape
tmp = np.column_stack((np.repeat(np.arange(m),n),np.tile(np.arange(n),m),fosMeans.reshape(m*n,-1)))
out_dfFos = pd.DataFrame(tmp,columns=colNames)
tmp = np.column_stack((np.repeat(np.arange(m),n),np.tile(np.arange(n),m),refMeans.reshape(m*n,-1)))
out_dfRef = pd.DataFrame(tmp,columns=colNames)
tmp = np.column_stack((np.repeat(np.arange(m),n),np.tile(np.arange(n),m),cortMeans.reshape(m*n,-1)))
out_dfCort = pd.DataFrame(tmp,columns=colNames)
tmp = np.column_stack((np.repeat(np.arange(m),n),np.tile(np.arange(n),m),fosNormMeans.reshape(m*n,-1)))
out_dfFosNorm = pd.DataFrame(tmp,columns=colNames)

In [ ]:
out_dfFosNorm.head()

In [ ]:
df_melt=out_dfFos.melt(['group','animal'], value_name='fos')
#df_melt.loc[df_melt.area==0,'area']=np.nan
#df_melt=df_melt[df_melt.fos!=0]

df_meltRef=out_dfRef.melt(['group','animal'], value_name='signal')
#df_meltRef=df_meltRef[df_meltRef.signal!=0]

df_meltCort=out_dfCort.melt(['group','animal'], value_name='signal')
#df_meltCort=df_meltCort[df_meltCort.signal!=0]

df_meltfosNorm=out_dfFosNorm.melt(['group','animal'], value_name='signal')
#df_meltfosNorm=df_meltfosNorm[df_meltfosNorm.signal!=0]

df_melt['ref']=df_meltRef.signal
df_melt['cort']=df_meltCort.signal
df_melt['fosPreNorm']=df_meltfosNorm.signal

In [ ]:
df_melt.head()

In [ ]:
def normMean(v):
    return v/np.nanmean(v)

In [ ]:
groups

In [ ]:
df_melt['fosNorm']=df_melt.groupby(['variable'])['fos'].apply(normMean)
df_melt['fosRefNorm']=df_melt.fos/df_melt.ref
df_melt['fosRefNormNorm']=df_melt.groupby(['variable'])['fosRefNorm'].apply(normMean)

df_melt['fosCortNorm']=df_melt.fos/df_melt.cort
df_melt['fosCortNormNorm']=df_melt.groupby(['variable'])['fosCortNorm'].apply(normMean)

df_melt['fosPreNormNorm']=df_melt.groupby(['variable'])['fosPreNorm'].apply(normMean)

df_melt['CortNorm']=df_melt.groupby(['variable'])['cort'].apply(normMean)
df_melt['CortRefNorm']=df_melt.cort/df_melt.ref
df_melt['CortRefNormNorm']=df_melt.groupby(['variable'])['CortRefNorm'].apply(normMean)
df_melt['condition']=np.array(groups)[df_melt.group.astype('int')]

df_melt.head()

In [ ]:
dfMaskComments=pd.read_excel(p_maskComments,
                             sheet_name='hcrMerge_cFos',
                            header=1)
dfMaskComments.head()

In [ ]:
df_melt['areaNum']=np.unique(df_melt.variable,return_inverse=True)[1]
df_melt['areaName']=dfMaskComments.area[df_melt['areaNum']].values
df_melt['areaShort']=dfMaskComments.areaShort[df_melt['areaNum']].values
df_melt['areaMini']=dfMaskComments.areaMini[df_melt['areaNum']].values
df_melt.head()

In [ ]:
areaNames=dfMaskComments['area']
values = np.unique(brightMask.ravel())
values=values[~np.isnan(values)][:-1]


In [ ]:
tmp=df_melt.groupby(['areaName','condition'],sort=False).fosRefNorm.mean()
tmp[areaNames[1]]

In [ ]:
tmp[areaNames[2]]

# visualization of cfos difference to no-stim
## start by sorting brain area mean cfos according to correlation with attraction
### (this is later re-sorted according to clustering)

In [ ]:
attract=np.array([0.005,0.03,0.22,0.28])

In [ ]:
from scipy import stats
corr=np.zeros((nAreas,6))
for i,a in enumerate(areaNames):
    x=tmp[a]
    s, itc, r, p, std = stats.linregress(attract,x)
    corr[i]=[i,s, itc, r, p, std]
dfCorr=pd.DataFrame(corr,columns=['i','slope','intercept','r_value','p_value','std'])
dfCorr['area']=areaNames

dfCorr['sr']=dfCorr.slope*np.abs(dfCorr.r_value)
dfCorr.head()



In [ ]:
dfCorr.loc[dfCorr.area=='Outer','sr']=-99

dfCorr.sort_values(by='sr',ascending=False)

In [ ]:
order=np.argsort(dfCorr.sr).values[::-1]
order

In [ ]:
tt=df_melt.condition.unique()
tt.sort()
tt

In [ ]:
df_melt.loc[df_melt.group==0,'animal']

In [ ]:
def groupCohen(x, cat='condition',test=2,colName='fosRefNorm'):
    dat=x.copy()
    dat.set_index(['animal', cat])

    catLevels = np.array(['noStim', 'cont', 'bout', 'conspecific'])
    res=[]
    for i,c in enumerate(catLevels[1:]):
        d2 = dat.loc[dat[cat] == c,colName]#iloc[:,2:]
        d1 = dat.loc[dat[cat] == 'noStim',colName]#iloc[:,2:]

        # calculate the size of samples
        n1, n2 = len(d1), len(d2)
        # calculate the variance of the samples
        s1, s2 = np.nanvar(d1, ddof=1), np.nanvar(d2, ddof=1)
        # calculate the pooled standard deviation
        s = np.sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
        # calculate the means of the samples
        u1, u2 = np.nanmean(d1), np.nanmean(d2)
        # calculate the effect size
        r=(u2 - u1) / s

        #print('d1:',d1.values,'d2:',d2.values,s1,s2,r)
        res.append(r)
    return np.array(res)

In [ ]:
def groupttest(x, cat='condition',test=2,colName='fosRefNorm'):
    dat=x.copy()
    dat.set_index(['animal', cat])

    catLevels = np.array(['noStim', 'cont', 'bout', 'conspecific'])
    res=[]
    for i,c in enumerate(catLevels[1:]):
        d2 = dat.loc[dat[cat] == c,colName]#iloc[:,2:]
        d1 = dat.loc[dat[cat] == 'noStim',colName]#iloc[:,2:]
        s,p=sta.ttest_ind(d1,d2)
        #print(c,d1.mean(),d2.mean(),d1.mean()-d2.mean(),p)
        res.append(p)
    return np.array(res)

In [ ]:
tmp=df_melt.groupby(['areaNum','condition'],sort=False).fosRefNorm.mean().reset_index()
tmp=tmp.pivot(index='areaNum',columns='condition',values='fosRefNorm')
areaNames=df_melt.groupby(['areaName'],sort=False).areaMini.first()
tmp.index=areaNames[tmp.index]
order2=np.argsort(dfCorr.sr.values)[::-1]

tmp=tmp.iloc[order2]
tmp=tmp[tmp.columns[[3,2,0,1]]]

In [ ]:
tmp.head()

In [ ]:
d1=[1,2,3,18,22]
d2=[17,18,19,20,21]
print(sta.ttest_ind(d1,d2))
print(sta.mannwhitneyu(d1,d2,alternative='less'))

In [ ]:
values = np.unique(brightMask[~np.isnan(brightMask)].ravel())
order=dfCorr[dfCorr.i.isin(values-1)].sort_values(by='sr').i.values.astype('int')[::-1]
areaNames=dfMaskComments['areaMini'].values#[order]
labelNames=dfMaskComments['areaShort'].values#[order]
labelPretty=[a+': '+b for a,b in zip(areaNames,labelNames)]
labelPretty

In [ ]:
tmp2=tmp.copy()
tmp2=tmp2-np.tile(tmp2.noStim,(4,1)).T#[:-1]
len(labelPretty)

In [ ]:
tmp3=tmp.copy()
tmp3=(tmp3-np.tile(tmp3.noStim,(4,1)).T)/(tmp3+np.tile(tmp3.noStim,(4,1)).T)
len(labelPretty)

In [ ]:

g1=3
g2=2
ix=(df_melt.areaMini=='nMLF')&(df_melt.group==g1)
tt=df_melt[ix]
d1=tt.fosRefNorm.values
ix=(df_melt.areaMini=='nMLF')&(df_melt.group==g2)
tt=df_melt[ix]
d2=tt.fosRefNorm.values
n1, n2 = len(d1), len(d2)
# calculate the variance of the samples
s1, s2 = np.nanvar(d1, ddof=1), np.nanvar(d2, ddof=1)
# calculate the pooled standard deviation
s = np.sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
# calculate the means of the samples
u1, u2 = np.nanmean(d1), np.nanmean(d2)
# calculate the effect size
(u2 - u1) / s


In [ ]:
tmp4=tmp3.copy()
tmp4.loc[:,['cont','bout','conspecific']]=np.vstack(df_melt.groupby(['areaNum'],sort=False).apply(groupCohen)[order2].values)
tmp4.head()

In [ ]:
order2

In [ ]:
df_melt2=df_melt[~np.isnan(df_melt.fos)]
np.sum(np.isnan(df_melt2.fosRefNorm))

In [ ]:
tmp5=tmp3.copy()
tmp5.loc[:,['cont','bout','conspecific']]=np.vstack(df_melt2.groupby(['areaNum'],sort=False).apply(groupttest)[order2].values)
tmp6=tmp5.copy()
tmp6.loc[tmp6.index=='o',:]=0
bins = np.array([-0.00001,.001,.01,.05,10])/3 #Correct for number of comparisons in each 
labels = [3,2,1,0]
stars=np.array(['','*','**','***',''])
for c in ['cont','bout','conspecific']:
    tmp6.loc[:,c]=pd.cut(tmp6.loc[:,c], bins=bins,labels=labels).astype('int')
    tmp6[tmp6<0]=4
tmp6.loc[:,'noStim']=4
tmp6.loc[:,:]=stars[tmp6.values]
tmp6.head(30)

In [ ]:
tmp5.to_csv(FigureDir+'FigSource1eb.csv')

In [ ]:
df_melt2.to_csv(FigureDir+'FigSource1e.csv')

In [ ]:
tmp4[:-1].to_csv(FigureDir+'FigSource1ec.csv')

In [ ]:
fig, ax = plt.subplots(figsize=(20/inToCm,4/inToCm))
ax=sns.heatmap(tmp4[:-1].T,
               square=True,
               cmap='coolwarm',
               center=0,
               vmin=-3,
               vmax=4,
               yticklabels=['No stim',
                           'Continuous',
                           'Bout-like',
                           'Conspecific'],
                annot=tmp6[:-1].T,
               fmt = '',
               annot_kws={'size':7,
                         'color':'k'},
               cbar_kws={'shrink':0.4,
                         'aspect':10,
                         'pad':.01,
                        'label': 'Effect size'})

cbar = ax.collections[0].colorbar
cbar.set_ticks([-3, 0, 4])
ax.set_ylabel('')
ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 8);

ax.set_xlabel('Activity Cluster');
figPath=FigureDir+'hcr_allQuantHeat_cohen.svg'
plt.savefig(figPath,bbox_inches='tight')

In [ ]:
#fig, ax = plt.subplots(figsize=(20/inToCm,4/inToCm))

cg=sns.clustermap(tmp4[:-1].T,
               row_cluster=False,
                 figsize=(20/inToCm,4/inToCm))

clusterOrder=cg.dendrogram_col.reordered_ind



In [ ]:
cg=sns.clustermap(tmp4[:-1].T,
               cmap='coolwarm',
               center=0,
               row_cluster=False,
               annot=tmp6[:-1].T,#.iloc[:,clusterOrder],
               fmt = '',
                xticklabels = 1,
               vmin=-3,
               vmax=4,
              figsize=(28/inToCm,6/inToCm),
                annot_kws={'size':10,
                         'color':'k'},
                 )

cg.ax_heatmap.axvline(5,ls=':',c='k')
cg.ax_heatmap.axvline(11,ls=':',c='k')
cg.ax_heatmap.axvline(18,ls=':',c='k')
cg.ax_heatmap.axvline(23,ls=':',c='k')
cg.ax_heatmap.text(0,-1.5,'Group:', fontsize = 10)
cg.ax_heatmap.text(4,-1.5,'1', fontsize = 10)
cg.ax_heatmap.text(10,-1.5,'2', fontsize = 10)
cg.ax_heatmap.text(15,-1.5,'3', fontsize = 10)
cg.ax_heatmap.text(22,-1.5,'4', fontsize = 10)
cg.ax_heatmap.text(28,-1.5,'5', fontsize = 10)
cg.cax.set_visible(False)

cbaxes = cg.ax_heatmap.figure.add_axes([0.91, .45, 0.01, 0.3]) 
cb = plt.colorbar(ax.get_children()[0], 
                  cax = cbaxes,
                 orientation='vertical')  
cbaxes.text(-4.5,4.5,'4',fontsize=10)
cbaxes.text(-4.5,-4.5,'-3',fontsize=10)
cbaxes.text(10,-3,'Effect size \n(Cohen\'s d)',fontsize=10,rotation=90)
cbar = ax.collections[0].colorbar
cbar.set_ticks([-3,4])
cbar.set_ticklabels([])
cg.ax_heatmap.set_ylabel('')
cg.ax_heatmap.set_xlabel('Activity cluster',fontsize=10)
cg.ax_heatmap.set_yticklabels(ax.get_ymajorticklabels(), fontsize = 10);
cg.ax_heatmap.set_xticklabels(cg.ax_heatmap.get_xticklabels(), fontsize = 10);
cg.ax_heatmap.yaxis.set_label_position("left")
cg.ax_heatmap.yaxis.tick_left()

figPath=FigureDir+'hcr_allQuantHeat_cohenCluster.svg'
plt.savefig(figPath,bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(18/inToCm,6/inToCm))

values = np.unique(brightMask[~np.isnan(brightMask)].ravel())

# get the colors of the values, according to the 
# colormap used by imshow
#colors = [im.cmap(im.norm(value)) for value in np.arange(33)]
# create a patch (proxy artist) for every color 

order=dfCorr[dfCorr.i.isin(values-1)].sort_values(by='sr').i.values.astype('int')[::-1]
areaNames=dfMaskComments['areaMini'].values[order]

plt.setp(ax.lines, zorder=100)
plt.setp(ax.collections, zorder=100, label="")

sns.barplot(data=df_melt,
              x='areaMini',
              hue='condition',
              y='fosRefNorm',
              #join=False,
              ax=ax,
              #dodge=0.5,
              zorder=-100,
              ci='sd',
              #color='k'
              #markers='.',
              palette=co,
              order=areaNames[clusterOrder],
            estimator=np.nanmedian,
            errwidth=1,
              )

sns.despine()
ax.set_xlabel('');
#$\it{c-fos/}$\n$\it{elavl3}$
ax.set_ylabel('$\it{c}$-$\it{fos\ /\ elavl3}$');
ax.set_ylim([0,1.25])
handles, labels = ax.get_legend_handles_labels()
labels=np.array(['Bout-like','Continuous','Conspecific','No stim'])[[3,1,0,2]]

l = plt.legend(handles[0:4], labels[0:4], title='',ncol=4,handletextpad=0,
               loc='center',bbox_to_anchor=(.5, .9),
              frameon=False)

plt.xticks(rotation=90)

figPath=FigureDir+'hcr_allQuant2_norm_sort.svg'
plt.savefig(figPath,bbox_inches='tight')
figPath=FigureDir+'hcr_allQuant2_norm_sort.png'
plt.savefig(figPath,bbox_inches='tight')

# Plot annotated average slices

In [ ]:
from matplotlib.colors import ListedColormap

anatomyColors=ListedColormap(sns.color_palette('tab20',33))

In [ ]:
def PlotAnnotatedSlice(img,ax=None,labels=None,clim=None,cmap=None,aspect=1.0,fontSize=6):
    sns.set_palette('tab20', n_colors=33)
    areas=np.unique(img[~np.isnan(img)])
    areaCenters=np.zeros((areas.shape[0],3))
    for i,a in enumerate(areas):
        areaCenters[i,0]=a
        areaCenters[i,1:]=list(nd.measurements.center_of_mass(img==a))
    
    if ax is not None:
        im=ax.imshow(img,clim=clim,cmap=cmap,alpha=.8,aspect=aspect)
        for a in areaCenters:
            #ax.text(a[2]-5,a[1],str(int(a[0]))+":"+labels[int(a[0])-1],fontSize=6)
            ax.text(a[2]-5,a[1],labels[int(a[0])-1],fontsize=fontSize)
        return im
    else:
        return areaCenters
    

In [ ]:
areaNames

In [ ]:
def drawArrow(ax,length=40,ori=(1,1),pos=(100,100),labels=['A','L'],fontSize=8,col='w'):
    ax.annotate( labels[0], xy=(pos[0]-(length*ori[0]), pos[1]),
              xytext=(pos[0]-(length*ori[0]), pos[1]-(length*ori[1])) , 
                va = "bottom", 
                ha="center",
                color=col,
                annotation_clip=False,
                fontsize=fontSize,
               arrowprops=dict(color=col,
                              arrowstyle='<-',
                               lw=1,
                              shrinkB=0),
                bbox=dict(pad=-5, facecolor="none", edgecolor="none")
               )
    ax.annotate( labels[1], xy=(pos[0]-(length*ori[0]), pos[1]),
              xytext=(pos[0], pos[1]) , 
                va = "center", 
                ha="center",
                color=col,
                annotation_clip=False,
                fontsize=fontSize,
               arrowprops=dict(color=col,
                              arrowstyle='<-',
                               lw=1,
                               shrinkA=0,
                              shrinkB=0),
                bbox=dict(pad=-0, facecolor="none", edgecolor="none")
               )

In [ ]:
originalUmpPx=1406/1950
scaling=1950/512
zScaleFactor=3/(originalUmpPx*scaling)
zScaleFactor

In [ ]:
lim=[(80,380),(490,50),(0,200)]
clim=[(1,33),(0,50),(0,255),(0,50),(254,255),(0,2.5)]

xlim=(30,530)
ylim=(40,410)
x,y,z=255,305,160

z0=120
z2=200
mid=230
fig = plt.figure(figsize=(8,8))
axes=[]
axes.append(fig.add_subplot(2, 3, 2))
axes.append(fig.add_subplot(4, 2, 5))
axes.append(fig.add_subplot(4, 2, 6))
axes.append(fig.add_subplot(2, 3, 1))
axes.append(fig.add_subplot(2, 3, 3))
#axes.append(fig.add_subplot(4, 1, 4))
axes=np.array(axes)

maskHalf=brightMask.copy()
maskHalf[:,:,:mid]=np.nan
fosmasked=np.nanmax(fosNorm_merge_avg,axis=0)*(masks_cFos!=14)
sns.set_palette('tab20', n_colors=33)
areaNames=dfMaskComments['areaMini'].values#[order]

axes[0].imshow(fosmasked[z],clim=clim[5],cmap='binary')
#axes[0].imshow(maskHalf[z],clim=clim[0],cmap=anatomyColors,alpha=.8)
PlotAnnotatedSlice(maskHalf[z],ax=axes[0],clim=clim[0],cmap=anatomyColors,labels=areaNames,fontSize=10)
axes[3].imshow(fosmasked[z0],clim=clim[5],cmap='binary')
#axes[3].imshow(maskHalf[z0],clim=clim[0],alpha=.8,cmap=anatomyColors)
PlotAnnotatedSlice(maskHalf[z0],ax=axes[3],clim=clim[0],cmap=anatomyColors,labels=areaNames,fontSize=10)
axes[4].imshow(fosmasked[z2],clim=clim[5],cmap='binary')

#axes[4].imshow(maskHalf[z2],clim=clim[0],alpha=.8,cmap=anatomyColors)
PlotAnnotatedSlice(maskHalf[z2],ax=axes[4],clim=clim[0],cmap=anatomyColors,labels=areaNames,fontSize=10)
im2=axes[2].imshow(fosmasked[:,y,:],clim=clim[5],cmap='binary',aspect=zScaleFactor)
#im=axes[2].imshow(maskHalf[:,y,:],clim=clim[0],alpha=.8,cmap=anatomyColors)
im=PlotAnnotatedSlice(maskHalf[:,y,:],ax=axes[2],clim=clim[0],cmap=anatomyColors,labels=areaNames,aspect=zScaleFactor,fontSize=10)
axes[1].imshow(fosmasked[:,:,x],clim=clim[5],cmap='binary',aspect=zScaleFactor)
#axes[1].imshow(maskHalf[:,:,x],clim=clim[0],alpha=.8,cmap=anatomyColors)
PlotAnnotatedSlice(maskHalf[:,:,x],ax=axes[1],clim=clim[0],cmap=anatomyColors,labels=areaNames,aspect=zScaleFactor,fontSize=10)

for a in axes[[0,3,4]]:
    a.axvline(x,c='lightgray')
    a.axhline(y,c='lightgray')
    a.axvline(mid,c='gray',ls='--')
    a.set_ylim(lim[1])
    a.set_xlim(lim[0])

axes[1].set_ylim([270,70])
axes[1].set_xlim([40,480])
axes[2].set_xlim(lim[0])
axes[2].set_ylim([270,70])


axes[3].text(lim[0][0],lim[1][1]+20,'XY',color='k',fontsize=10)
axes[3].text(lim[0][0],lim[1][1]+60,'z='+str(int(z0*3.))+' um',color='k',fontsize=10)
axes[0].text(lim[0][0],lim[1][1]+20,'XY',color='k',fontsize=10)
axes[0].text(lim[0][0],lim[1][1]+60,'z='+str(int(z*3.))+' um',color='k',fontsize=10)
axes[4].text(lim[0][0],lim[1][1]+20,'XY',color='k',fontsize=10)
axes[4].text(lim[0][0],lim[1][1]+60,'z='+str(int(z2*3.))+' um',color='k',fontsize=10)
axes[1].text(lim[0][0],lim[1][1]+20,'YZ',color='k',fontsize=10)
axes[2].text(lim[0][0],lim[1][1]+20,'XZ',color='k',fontsize=10)

axes[1].axvline(y,c='lightgray')
axes[1].axhline(z,c='lightgray')
axes[1].axhline(z0,c='lightgray')
axes[1].axhline(z2,c='lightgray')

axes[2].axvline(x,c='lightgray')
axes[2].axhline(z,c='lightgray')
axes[2].axhline(z0,c='lightgray')
axes[2].axhline(z2,c='lightgray')
axes[2].axvline(mid,c='gray',ls='--')


    
for i,a in enumerate(axes):
    a.axis('off')
    a.set_xticks([])
    a.set_yticks([])
    a.set_ylabel("")
    a.set_position([0, 0, 1, 1], which='both')
    if i==4:
        x=[lim[0][0],lim[0][0]+200/2.75]
        y=[lim[1][0]+10,lim[1][0]+10]
        line = lines.Line2D(x, y, lw=2, color='k', alpha=1,solid_capstyle='butt')
        line.set_clip_on(False)
        a.add_line(line)

        
drawArrow(axes[4],pos=(330,490),labels=['A','L'],ori=(-1,1),length=50,col='k',fontSize=10)
drawArrow(axes[1],pos=(450,240),labels=['D','A'],ori=(-1,1),length=50,col='k',fontSize=10)
drawArrow(axes[2],pos=(330,240),labels=['D','L'],ori=(-1,1),length=50,col='k',fontSize=10)

cax = fig.add_axes([0.86, 0.35, 0.01, 0.05])
clb=fig.colorbar(im2, cax=cax,
             orientation='vertical',
            pad=0.1,
                           shrink=.5,
            )

clb.ax.tick_params(labelsize=10)
clb.ax.text(-.5,3.2,'$\it{c}$-$\it{fos}$/\n$\it{elavl3}$',fontsize=10)
      
        
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=None)
#fig.tight_layout(pad=0)
figPath=FigureDir+'hcr_cfosMasksAndNames_3h1c1s_clean.svg'
plt.savefig(figPath,bbox_inches='tight')

In [ ]:
from skimage import exposure
refAllMergeGamma=exposure.adjust_gamma(refAllMerge, 0.5)

In [ ]:
lim=[(80,380),(490,50),(0,200)]
clim=[(1,33),(0,50),(0,255),(0,50),(253,255)]

xlim=(30,530)
ylim=(40,410)

zTop=115
zBottom=240

yFront=100
yBack=500
nPerView=12

fig,axes=plt.subplots(6,4,figsize=(15/inToCm,27/inToCm), dpi=300,
                     gridspec_kw={'height_ratios': [4,4,4,3,3,3]})
axes=axes.ravel()

maskHalf=brightMask.copy()
maskHalf[:,:,:mid]=np.nan
fosmasked=np.nanmax(fosNorm_merge_avg,axis=0)*(masks_cFos!=14)
sns.set_palette('tab20', n_colors=33)

for i in range(nPerView):
    zDiff=(zBottom-zTop)/nPerView
    z=int(zTop+(zDiff*i))
    axes[i].imshow(255-refAllMergeGamma[z],clim=(246,255),cmap='gray')
    axes[i].text(lim[0][0],lim[1][1],'z='+str(z*3)+'um',fontsize=8)
    PlotAnnotatedSlice(maskHalf[z],ax=axes[i],clim=clim[0],cmap=anatomyColors,labels=areaNames)

    axes[i].set_ylim(lim[1])
    axes[i].set_xlim(lim[0])
    if i==nPerView-1:
        x=[lim[0][1]-200/2.75,lim[0][1]]
        y=[lim[1][0]+10,lim[1][0]+10]
        line = lines.Line2D(x, y, lw=2, color='k', alpha=1)
        line.set_clip_on(False)
        axes[i].add_line(line)
        drawArrow(axes[i],pos=(250,450),labels=['A','L'],ori=(-1,1),length=100,col='k')
    
for i in range(nPerView):
    yDiff=(yBack-yFront)/nPerView
    y=int(yFront+(yDiff*i))
    axes[i+nPerView].imshow(255-refAllMergeGamma[:,y,:],clim=(246,255),cmap='gray',aspect=zScaleFactor)
    axes[i+nPerView].text(lim[0][0],-20,'y='+str(y*3)+'um',fontsize=8)

    PlotAnnotatedSlice(maskHalf[:,y,:],ax=axes[i+nPerView],clim=clim[0],cmap=anatomyColors,labels=areaNames,aspect=zScaleFactor)

    axes[i+nPerView].set_xlim(lim[0])
    axes[i+nPerView].set_ylim([270,0])

fig.text(0,1,'Figure S1')
for i,a in enumerate(axes.ravel()):
    a.axis('off')
    a.set_xticks([])
    a.set_yticks([])
    a.set_ylabel("")
    #a.set_position([0, 0, 1, 1], which='both')
    
#drawArrow(axes[4],pos=(230,400),labels=['A','L'],ori=(1,1),length=200)


fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=None)
fig.tight_layout(pad=1)
figPath=FigureDir+'FigureS1_hcr_refMasksAndNames.svg'
plt.savefig(figPath,bbox_inches='tight')

In [ ]:
zoomCoors=np.array([
    [60,250,360,160,115,125,0,0.5,[18,16],2],
    [160,250,140,100,148,180,0,1,[1,2],1.5],
    [170,295,120,130,190,210,0,1,[4,21,6],1.5]
])
print(zoomCoors)
print(zoomCoors.shape)

In [ ]:
    
nGroups=len(groups)
nTiles=zoomCoors.shape[0]
fig = plt.figure(figsize=(12, 12), dpi=200)

gs = gridspec.GridSpec(nrows=nTiles, ncols=5, width_ratios=[1, 1, 1, 1, .1])

In [ ]:
def getContours(mask,zoomCoors):
    image= mask[0]*0
    #image=image.astype('uint8')*255
    image=cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    centroids=[]
    for area in zoomCoors[8]:
        s1=zoomCoors[4]#.astype('int')
        s2=zoomCoors[5]#.astype('int')
        mid=int((s1+s2)/2.)
        tmp=mask==area
        #maskMax=tmp.max(axis=0).astype('uint8')*255
        maskMax=tmp[mid].astype('uint8')*255
        _, binary = cv2.threshold(255-maskMax, 225, 255, cv2.THRESH_BINARY_INV)
        contours, hierarchy = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        image = cv2.drawContours(image, contours, -1, (0, 255, 0), int(zoomCoors[9]))

        for c in contours:
            # compute the center of the contour
            M = cv2.moments(c)
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            centroids.append((cX,cY))
    return image,centroids

In [ ]:
def annotateRegion(ax,mask,zoomCoors,labels):
    image= mask[0]*0
    for area in zoomCoors[8]:
        ax.tex

In [ ]:
import matplotlib.cm as cm
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
def plot_zoomTiles(img,
                         groups,
                         zoomCoors,
                  ):
    
    nGroups=len(groups)
    nTiles=zoomCoors.shape[0]
    figRat=4/np.sum(zoomCoors[:,3]/zoomCoors[:,2])
    figW=8
    fig = plt.figure(figsize=(figW, figW/figRat))

    gs = gridspec.GridSpec(nrows=nTiles, 
                           ncols=4, 
                           width_ratios=[1, 1, 1, 1],
                           height_ratios=zoomCoors[:,3]/zoomCoors[:,2])
    gs.update(wspace=0.02, hspace=0.02)
    my_cmap = cm.gray
    my_cmap.set_under('k', alpha=0)
    cmap_cort = 'inferno'
    axs=[]
    
    for tile in range(nTiles):
        for gno, group in enumerate(groups):
            s1=zoomCoors[tile,4]#.astype('int')
            s2=zoomCoors[tile,5]#.astype('int')
            x1,w=zoomCoors[tile,0],zoomCoors[tile,2]
            y1,h=zoomCoors[tile,1],zoomCoors[tile,3]
            
            xlim=(x1,x1+w)
            ylim=(y1,y1+h)
            axs.append(plt.subplot(gs[tile, gno]))
            axs[-1].imshow(img[gno,s1:s2].mean(axis=0), 
                           clim=[zoomCoors[tile,6],
                                 zoomCoors[tile,7]], 
                           cmap='inferno')
            contours,centroids=getContours(masks_cFos,zoomCoors[tile])
            contours=contours.astype('float')/255
            #contours[np.where(contours==0)]=np.nan
            #maskedContours=np.ma.masked_where(contours < 1, contours)
            axs[-1].imshow(contours[:,:,1],cmap=my_cmap,clim=[0.1, 1.2])
                
            axs[-1].set_axis_off()
            axs[-1].set_xlim(xlim)
            axs[-1].set_ylim(ylim[::-1])
            if tile==0:
                x,y = np.array([[x1+w,x1], [y1-10,y1-10]])
                line = lines.Line2D(x, y, lw=3, color=co[gno], alpha=1,solid_capstyle='butt')
                line.set_clip_on(False)
                axs[-1].add_line(line)
                axs[-1].set_title(titles[gno],fontsize=10)
                
            if gno==0:
                x=[10+x1,10+x1+200/2.75]
                y=[y1+h-4,y1+h-4]
                line = lines.Line2D(x, y, lw=2, color='w', alpha=1,solid_capstyle='butt')
                line.set_clip_on(False)
                axs[-1].add_line(line)
        

        axins = inset_axes(axs[-1],
                   width="5%",  # width = 5% of parent_bbox width
                   height="90%",  # height : 50%
                   loc='lower left',
                   bbox_to_anchor=(1.05, 0., 1, 1),
                   bbox_transform=axs[-1].transAxes,
                   borderpad=0,
                   )

        sm = plt.cm.ScalarMappable(cmap='inferno', 
                                   norm=plt.Normalize(vmin=zoomCoors[tile,6], 
                                                      vmax=zoomCoors[tile,7]))
        sm._A = []
        clb = plt.colorbar(sm, 
                           pad=0.1,
                           shrink=.5,
                           cax=axins,ticks=[0,.2])
        #clb.ax.tick_params(labelsize=10)
        #clb.set_label('cFOS',fontsize=8)
        

    
    for i,a in enumerate(axs):
        a.axis('off')
        a.set_xticks([])
        a.set_yticks([])
        a.set_ylabel("")
        #a.set_position([0, 0, 1, 1], which='both')

    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=5, hspace=5)
    fig.tight_layout(pad=5)

    

    return fig,axs,centroids

In [ ]:
miniLabels=dfMaskComments.areaMini.values

miniLabels

In [ ]:
miniLabels=dfMaskComments.areaMini.values
fig,axs,centroids=plot_zoomTiles(fosNorm_merge_avg*(masks_cFos!=14),groups,zoomCoors)

axs[0].text(165,405,miniLabels[zoomCoors[0][8][0]-1],size=10,color='w')
axs[0].text(210,370,miniLabels[zoomCoors[0][8][1]-1],size=10,color='w')

axs[4].text(170,310,miniLabels[zoomCoors[1][8][0]-1],size=10,color='w')
axs[4].text(170,280,miniLabels[zoomCoors[1][8][1]-1],size=10,color='w')

axs[8].text(180,330,miniLabels[zoomCoors[2][8][0]-1],size=10,color='w')
axs[8].text(180,370,miniLabels[zoomCoors[2][8][1]-1],size=10,color='w')
axs[8].text(180,410,miniLabels[zoomCoors[2][8][2]-1],size=10,color='w')
plt.rcParams['svg.fonttype'] = 'none'
drawArrow(axs[7],pos=(250,340),labels=['A','L'],ori=(-1,1),fontSize=10)

figPath=FigureDir+'hcr_cfosExamplePlanes.svg'
plt.savefig(figPath,bbox_inches='tight')

In [ ]:
centroids


im_refCortFos=mpimg.imread('J:\\_Projects\\bpn_man\\data\\raw\\hcr_allAnimalAvg_ref_cort_fos-160.png')

In [ ]:
dfCellCounts=pd.read_excel(p_cellCounts,header=0)
dfCellCounts.head()

In [ ]:
tmp=pd.melt(dfCellCounts.loc[:7,['Cort','CortFos']],var_name='marker',value_name='count')
tmp.loc[:,'group']='merge'
tmp2=dfCellCounts[['condition','fosR']]
tmp2.loc[:,'fosR']=tmp2.fosR.values*2
tmp2.columns=['group','count']
tmp2['marker']='cFos'
tmp3=pd.concat([tmp,tmp2],sort=True)

In [ ]:
co2=co
co2.append([0,0,0])
co2=np.array(co2)[[0,1,4,2,3]]
co2

In [ ]:
tmp3.reset_index()

In [ ]:
sns.pointplot(data=tmp3.reset_index(),
              x='marker',
              hue='group',
              y='count')

In [ ]:

plt.rcParams['svg.fonttype'] = 'path'
plt.rcParams['font.family'] = "sans-serif"
plt.rcParams['font.sans-serif'] = "Arial"

fig,ax=plt.subplots(figsize=(3,3),sharey=True)

sns.pointplot(data=tmp3,
              x='marker',
              hue='group',
              y='count',
              order=['cFos','Cort','CortFos'],
              hue_order=['no','cont','merge','bout','conspecific'],
             join=False,
              palette=co2,
             dodge=1,
             ci='sd');


sns.swarmplot(data=tmp3,
              x='marker',
              hue='group',
              y='count',
              order=['cFos','Cort','CortFos'],
              hue_order=['no','cont','merge','bout','conspecific'],
             #join=False,
              palette=co2,
             dodge=1);

plt.ylabel('Cell count DT')
sns.despine()
plt.xlim([-1,2.8])

handles, labels = ax.get_legend_handles_labels()
handles=np.array(handles)
labels=np.array(['Bout-like','Continuous','Conspecific','No stim','Combined'])[[3,1,0,2,4]]

l = plt.legend(handles[[0,1,3,4,2]], labels, title='',ncol=3,handletextpad=0,
               bbox_to_anchor=(1, 1.15),
               labelspacing=0,
               borderpad=0,
               handlelength=1.5,
               columnspacing=0.1,
              frameon=0,
              fontsize=8
              )
plt.axhline(0,c='gray',ls=':')
plt.axvline(0.75,c='gray',ls=':')
plt.xlabel('')
axlab=ax.get_xticklabels()
#$\it{c-fos/}$\n$\it{elavl3}$
from matplotlib import rc

rc('text', usetex=False)

#axlab[0]='\textbf{c-fos}'

axlab[0]='$\it{c}$-$\it{fos}$'
axlab[1]='$\it{cort}$'
axlab[2]='$\it{cort}$ & $\it{c}$-$\it{fos}$\nco-expressed'
ax.set_xticklabels(axlab, rotation = 45);
ax.set_yticks([0,100]);
ax.set_yticklabels(['0',' 100'],rotation=90);
figPath=FigureDir+'hcr_cfosDTcounts.svg'
#plt.rcParams['svg.fonttype'] = 'none'

plt.savefig(figPath,bbox_inches='tight')


In [ ]:
sns.swarmplot(data=tmp3,
              x='marker',
              hue='group',
              y='count',
              order=['cFos','Cort','CortFos'],
              hue_order=['no','cont','merge','bout','conspecific'],
             #join=False,
              palette=co2,
             dodge=1);

In [ ]:
tmp3.to_csv(FigureDir+'figSourceS2b.csv')

clim=[(0,255),(0,10),(0,255),(0,60),(0,0.5),(0,100),(254.5,255)]

fig = plt.figure(figsize=(3,3),
                     #constrained_layout=True,
                )
axes=[]
axes.append(0)
axes.append(0)
axes.append(0)

for i in np.arange(4):
    axes.append(fig.add_subplot(2, 2, i+1))

       
axes=np.array(axes)

z1=zoomCoors[1,4]#.astype('int')
z2=zoomCoors[1,5]#.astype('int')
midz=int((z1+z2)/2.)
midz=160

x1=zoomCoors[1,0]+20#.astype('int')
wx=zoomCoors[1,2]-40#.astype('int')
midx=int((x1+(wx/2.)))
midx=213

y1=zoomCoors[1,1]#.astype('int')
wy=zoomCoors[1,3]#.astype('int')
midy=int((y1+(wy/2.)))
midy=305

#np.nanmax(fosNorm_merge_avg,axis=0)

fosmasked=np.nanmax(fosNorm_merge_avg,axis=0)*(masks_cFos!=14)

im_h=255-fosmasked[midz]
im_s=255-fosmasked[:,:,midx]
im_c=255-fosmasked[:,midy,:]

my_cmap = cm.gray
my_cmap.set_under('k', alpha=1)
    

         
axes[4].imshow(fosNorm_merge_avgmax[midz],clim=clim[5],cmap='gray')
axes[3].imshow(refAllMerge[midz],clim=clim[1],cmap='gray')
axes[5].imshow(cort_merge_avg[midz],clim=clim[3],cmap='gray')
axes[6].imshow(im_refCortFos)     


axes[3].text(x1+10,y1+15,'$\it{elavl3}$',color='w',fontsize=10)
axes[4].text(x1+10,y1+15,'$\it{c}$-$\it{fos}$',color='w',fontsize=10)
axes[5].text(x1+10,y1+15,'$\it{cort}$',color='w',fontsize=10)

axes[6].text(x1+3,y1+15,'$\it{elavl3}$',color='white',fontsize=10)
axes[6].text(x1+50,y1+15,'$\it{c}$-$\it{fos}$',color='yellow',fontsize=10)
axes[6].text(x1+3,y1+30,'$\it{cort}$',color='magenta',fontsize=10)



for a in axes[[3,4,5,6]]:
    a.set_xlim([x1,x1+wx])
    


for a in axes[[3,4,5,6]]:
    a.set_ylim([y1+wy,y1])
                

for i,a in enumerate(axes[3:]):
    a.axis('off')
    a.set_xticks([])
    a.set_yticks([])
    a.set_ylabel("")
    a.set_position([0, 0, 1, 1], which='both')
    if i==2:
        x=[10+x1,10+x1+100/2.75]
        y=[y1+wy-4,y1+wy-4]
        line = lines.Line2D(x, y, lw=2, color='w', alpha=1,solid_capstyle='butt')
        line.set_clip_on(False)
        a.add_line(line)

fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=None)
fig.tight_layout(pad=.2)
plt.rcParams['svg.fonttype'] = 'path'
figPath=FigureDir+'hcr_cfosOrthoAndOverlay.svg'
plt.savefig(figPath,bbox_inches='tight')